In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from functions import *

pd.set_option('display.max_columns', 100)

In [2]:
def process_installment(ins):

    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

    # Group payments and get Payment difference
    ins = do_sum(ins, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')
    ins['PAYMENT_DIFFERENCE'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT_GROUPED']
    ins['PAYMENT_RATIO'] = ins['AMT_INSTALMENT'] / ins['AMT_PAYMENT_GROUPED']
    ins['PAID_OVER_AMOUNT'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
    ins['PAID_OVER'] = (ins['PAID_OVER_AMOUNT'] > 0).astype(int)

    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

    # Days past due and days before due (no negative values)
    ins['DPD_diff'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD_diff'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD_diff'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD_diff'].apply(lambda x: x if x > 0 else 0)

    # Flag late payment
    ins['LATE_PAYMENT'] = ins['DBD'].apply(lambda x: 1 if x > 0 else 0)
    ins['INSTALMENT_PAYMENT_RATIO'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['LATE_PAYMENT_RATIO'] = ins.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

    # Flag late payments that have a significant amount
    ins['SIGNIFICANT_LATE_PAYMENT'] = ins['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
    
    # Flag k threshold late payments
    ins['DPD_7'] = ins['DPD'].apply(lambda x: 1 if x >= 7 else 0)
    ins['DPD_15'] = ins['DPD'].apply(lambda x: 1 if x >= 15 else 0)

    ins['INS_IS_DPD_UNDER_120'] = ins['DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    ins['INS_IS_DPD_OVER_120'] = ins['DPD'].apply(lambda x: 1 if (x >= 120) else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum', 'var'],
        'DBD': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum', 'min'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum', 'min'],
        'SK_ID_PREV': ['size', 'nunique'],
        'PAYMENT_DIFFERENCE': ['mean'],
        'PAYMENT_RATIO': ['mean', 'max'],
        'LATE_PAYMENT': ['mean', 'sum'],
        'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
        'LATE_PAYMENT_RATIO': ['mean'],
        'DPD_7': ['mean'],
        'DPD_15': ['mean'],
        'PAID_OVER': ['mean'],
        'DPD_diff':['mean', 'min', 'max'],
        'DBD_diff':['mean', 'min', 'max'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']
    }

    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

    # from oof (DAYS_ENTRY_PAYMENT)
    cond_day = ins['DAYS_ENTRY_PAYMENT'] >= -365
    ins_d365_grp = ins[cond_day].groupby('SK_ID_CURR')
    ins_d365_agg_dict = {
        'SK_ID_CURR': ['count'],
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'sum'],
        'DAYS_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_INSTALMENT': ['mean', 'max', 'sum'],
        'AMT_PAYMENT': ['mean', 'max', 'sum'],
        'PAYMENT_DIFF': ['mean', 'min', 'max', 'sum'],
        'PAYMENT_PERC': ['mean', 'max'],
        'DPD_diff': ['mean', 'min', 'max'],
        'DPD': ['mean', 'sum'],
        'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
        'INS_IS_DPD_OVER_120': ['mean', 'sum']}

    ins_d365_agg = ins_d365_grp.agg(ins_d365_agg_dict)
    ins_d365_agg.columns = ['INS_D365' + ('_').join(column).upper() for column in ins_d365_agg.columns.ravel()]

    ins_agg = ins_agg.merge(ins_d365_agg, on='SK_ID_CURR', how='left')

    print('"Installments Payments" final shape:', ins_agg.shape)
    return ins_agg

raw

In [3]:
installments = pd.read_csv('raw-data/dseb63_installments_payments.csv')
installments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR
0,1054186,1.0,6,-1180.0,-1187.0,6948.360,6948.360,147397.0
1,2452854,1.0,21,-546.0,-552.0,11302.605,11302.605,147397.0
2,1054186,1.0,2,-1300.0,-1307.0,6948.360,6948.360,147397.0
3,1682318,1.0,2,-240.0,-243.0,7374.510,7374.510,147397.0
4,2452854,1.0,10,-876.0,-882.0,11302.605,11302.605,147397.0


eda

In [4]:
installments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138490 entries, 0 to 2138489
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   NUM_INSTALMENT_VERSION  float64
 2   NUM_INSTALMENT_NUMBER   int64  
 3   DAYS_INSTALMENT         float64
 4   DAYS_ENTRY_PAYMENT      float64
 5   AMT_INSTALMENT          float64
 6   AMT_PAYMENT             float64
 7   SK_ID_CURR              float64
dtypes: float64(6), int64(2)
memory usage: 130.5 MB


In [5]:
installments.describe()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR
count,2.138490e+06,2.138490e+06,2.138490e+06,2.138490e+06,2.138082e+06,2.138490e+06,2.138081e+06,2.138489e+06
mean,1.900536e+06,7.532254e-01,2.322613e+01,-1.104810e+03,-1.112679e+03,1.621761e+04,1.623699e+04,1.542862e+05
std,5.359076e+05,9.338448e-01,3.004550e+01,8.031695e+02,8.030702e+02,4.903047e+04,5.289689e+04,8.857895e+04
min,1.000011e+06,0.000000e+00,1.000000e+00,-2.922000e+03,-3.115000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.434096e+06,0.000000e+00,5.000000e+00,-1.745000e+03,-1.753000e+03,3.425985e+03,2.790585e+03,7.735300e+04
50%,1.895056e+06,1.000000e+00,1.000000e+01,-9.200000e+02,-9.280000e+02,8.167455e+03,7.473690e+03,1.545510e+05
75%,2.365976e+06,1.000000e+00,2.800000e+01,-4.110000e+02,-4.190000e+02,1.609704e+04,1.521549e+04,2.302120e+05
max,2.843477e+06,4.200000e+01,2.440000e+02,-2.000000e+00,-2.000000e+00,3.371884e+06,3.371884e+06,3.075080e+05


In [6]:
# # plot correlation matrix
# corr = installments.corr()
# plt.figure(figsize=(10, 10))
# sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
# plt.show()

In [7]:
# # plot histogram
# for col in installments.columns:
#     if col not in ['SK_ID_CURR', 'TARGET', 'SK_ID_PREV']:
#         fig, ax = plt.subplots(1, 2, figsize=(12, 4))
#         ax[0].hist(installments[col], bins=30)
#         ax[0].set_title(col)
#         ax[1].scatter(installments[col], installments['TARGET'])
#         ax[1].set_title(col)
#         plt.show()

fill na

In [8]:
# fill missing values with mode
for col in installments.columns:
    installments[col].fillna(installments[col].mode()[0], inplace=True)

installments.isnull().sum()

SK_ID_PREV                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
SK_ID_CURR                0
dtype: int64

In [9]:
# # plot histogram hue by target
# for col in installments.select_dtypes('number').columns:
#     if not col in ['SK_ID_CURR', 'TARGET', 'SK_ID_PREV']:
#         plt.figure(figsize=(12, 6))
#         sns.kdeplot(installments.loc[installments['TARGET'] == 0, col], label='target == 0')
#         sns.kdeplot(installments.loc[installments['TARGET'] == 1, col], label='target == 1')
#         plt.xlabel(col)
#         plt.ylabel('Density')
#         plt.title(f'{col} Distribution')
#         plt.legend()
#         plt.show()

feature engineering

In [10]:
# Change in Installment Version
installments['VERSION_CHANGE'] = installments.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].diff().fillna(0)

# Installment Timing
installments['TIMING_DIFF'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']

# Payment Ratio
installments['PAYMENT_RATIO'] = installments['AMT_PAYMENT'] / installments['AMT_INSTALMENT']

# Trend in Payment Amounts
installments['MOVING_AVG_PAYMENT'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Cumulative Features
installments['TOTAL_PAID_SO_FAR'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].cumsum()

# Payment Regularity
installments['PAYMENT_REGULARITY'] = installments.groupby('SK_ID_PREV')['TIMING_DIFF'].transform('std')

# Delayed Payment Count
installments['DELAYED_PAYMENT_COUNT'] = (installments['TIMING_DIFF'] > 0).astype(int).groupby(installments['SK_ID_PREV']).cumsum()

# Interaction Features
installments['VERSION_PAYMENT_INTERACTION'] = installments['NUM_INSTALMENT_VERSION'] * installments['AMT_PAYMENT']

# Categorical Encoding of Version Changes
installments['VERSION_CHANGE_CAT'] = pd.Categorical(installments['VERSION_CHANGE'])


# Assuming installments is your DataFrame and DAYS_INSTALMENT is the relative day column
# Convert DAYS_INSTALMENT to a cumulative count
max_day = installments['DAYS_INSTALMENT'].abs().max()
installments['DAY_COUNT'] = max_day - installments['DAYS_INSTALMENT'].abs()

# Sort by this new count
installments = installments.sort_values(by=['SK_ID_PREV', 'DAY_COUNT'])

# Calculate the sum of payments for the last 180 days
# Here, 180 days is just an example, adjust as needed
installments['SUM_LAST_180_DAYS'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].rolling(window=180, min_periods=1).sum().reset_index(level=0, drop=True)

installments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR,VERSION_CHANGE,TIMING_DIFF,PAYMENT_RATIO,MOVING_AVG_PAYMENT,TOTAL_PAID_SO_FAR,PAYMENT_REGULARITY,DELAYED_PAYMENT_COUNT,VERSION_PAYMENT_INTERACTION,VERSION_CHANGE_CAT,DAY_COUNT,SUM_LAST_180_DAYS
1132295,1000011,1.0,1,-435.0,-438.0,92435.04,92435.04,268838.0,0.0,-3.0,1.0,92435.04,1109158.65,1.969464,2,92435.04,0.0,2487.0,92435.04
1132292,1000011,1.0,2,-405.0,-411.0,92435.04,92435.04,268838.0,0.0,-6.0,1.0,92435.04,1016723.61,1.969464,2,92435.04,0.0,2517.0,184870.08
1132287,1000011,1.0,3,-375.0,-377.0,92435.04,92435.04,268838.0,0.0,-2.0,1.0,92414.43,646983.45,1.969464,1,92435.04,0.0,2547.0,277305.12
1132281,1000011,1.0,4,-345.0,-346.0,92435.04,92435.04,268838.0,0.0,-1.0,1.0,92435.04,369740.16,1.969464,1,92435.04,0.0,2577.0,369740.16
1132275,1000011,1.0,5,-315.0,-314.0,92435.04,92435.04,268838.0,0.0,1.0,1.0,92435.04,92435.04,1.969464,1,92435.04,0.0,2607.0,462175.20


In [11]:
# fill na with mode
for col in installments.columns:
    installments[col].fillna(installments[col].mode()[0], inplace=True)

installments.isnull().sum()

SK_ID_PREV                     0
NUM_INSTALMENT_VERSION         0
NUM_INSTALMENT_NUMBER          0
DAYS_INSTALMENT                0
DAYS_ENTRY_PAYMENT             0
AMT_INSTALMENT                 0
AMT_PAYMENT                    0
SK_ID_CURR                     0
VERSION_CHANGE                 0
TIMING_DIFF                    0
PAYMENT_RATIO                  0
MOVING_AVG_PAYMENT             0
TOTAL_PAID_SO_FAR              0
PAYMENT_REGULARITY             0
DELAYED_PAYMENT_COUNT          0
VERSION_PAYMENT_INTERACTION    0
VERSION_CHANGE_CAT             0
DAY_COUNT                      0
SUM_LAST_180_DAYS              0
dtype: int64

aggregate

In [12]:
installments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2138490 entries, 1132295 to 1054068
Data columns (total 19 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   SK_ID_PREV                   int64   
 1   NUM_INSTALMENT_VERSION       float64 
 2   NUM_INSTALMENT_NUMBER        int64   
 3   DAYS_INSTALMENT              float64 
 4   DAYS_ENTRY_PAYMENT           float64 
 5   AMT_INSTALMENT               float64 
 6   AMT_PAYMENT                  float64 
 7   SK_ID_CURR                   float64 
 8   VERSION_CHANGE               float64 
 9   TIMING_DIFF                  float64 
 10  PAYMENT_RATIO                float64 
 11  MOVING_AVG_PAYMENT           float64 
 12  TOTAL_PAID_SO_FAR            float64 
 13  PAYMENT_REGULARITY           float64 
 14  DELAYED_PAYMENT_COUNT        int64   
 15  VERSION_PAYMENT_INTERACTION  float64 
 16  VERSION_CHANGE_CAT           category
 17  DAY_COUNT                    float64 
 18  SUM_LAST_180_DAYS    

In [13]:
aggregations_1 = {
    'SK_ID_CURR': ['first'],
    'NUM_INSTALMENT_VERSION': ['mean', 'sum', 'std'],
    'NUM_INSTALMENT_NUMBER': ['mean', 'sum', 'std'],
    'DAYS_INSTALMENT': ['mean', 'sum', 'std'],
    'DAYS_ENTRY_PAYMENT': ['mean', 'sum', 'std'],
    'AMT_INSTALMENT': ['mean', 'sum', 'std'],
    'AMT_PAYMENT': ['mean', 'sum', 'std'],
    'VERSION_CHANGE': ['mean', 'sum', 'std'],
    'TIMING_DIFF': ['mean', 'sum', 'std', 'max'],
    'PAYMENT_RATIO': ['mean', 'sum', 'std', 'min'],
    'MOVING_AVG_PAYMENT': ['mean', 'sum', 'std'],
    'TOTAL_PAID_SO_FAR': ['mean', 'sum', 'std'],
    'PAYMENT_REGULARITY': ['mean', 'sum', 'std'],
    'DELAYED_PAYMENT_COUNT': ['mean', 'sum', 'std'],
    'VERSION_PAYMENT_INTERACTION': ['mean', 'sum', 'std'],
    'VERSION_CHANGE_CAT': ['nunique'],
    'DAY_COUNT': ['mean', 'sum', 'std'],
    'SUM_LAST_180_DAYS': ['mean', 'sum', 'std']
}

installments_agg_1 = installments.groupby('SK_ID_PREV').agg(aggregations_1)
installments_agg_1.columns = pd.Index([e[0] + "_" + e[1].upper() for e in installments_agg_1.columns.tolist()])
installments_agg_1.head()

,SK_ID_CURR_FIRST,NUM_INSTALMENT_VERSION_MEAN,NUM_INSTALMENT_VERSION_SUM,NUM_INSTALMENT_VERSION_STD,NUM_INSTALMENT_NUMBER_MEAN,NUM_INSTALMENT_NUMBER_SUM,NUM_INSTALMENT_NUMBER_STD,DAYS_INSTALMENT_MEAN,DAYS_INSTALMENT_SUM,DAYS_INSTALMENT_STD,DAYS_ENTRY_PAYMENT_MEAN,DAYS_ENTRY_PAYMENT_SUM,DAYS_ENTRY_PAYMENT_STD,AMT_INSTALMENT_MEAN,AMT_INSTALMENT_SUM,AMT_INSTALMENT_STD,AMT_PAYMENT_MEAN,AMT_PAYMENT_SUM,AMT_PAYMENT_STD,VERSION_CHANGE_MEAN,VERSION_CHANGE_SUM,VERSION_CHANGE_STD,TIMING_DIFF_MEAN,TIMING_DIFF_SUM,TIMING_DIFF_STD,TIMING_DIFF_MAX,PAYMENT_RATIO_MEAN,PAYMENT_RATIO_SUM,PAYMENT_RATIO_STD,PAYMENT_RATIO_MIN,MOVING_AVG_PAYMENT_MEAN,MOVING_AVG_PAYMENT_SUM,MOVING_AVG_PAYMENT_STD,TOTAL_PAID_SO_FAR_MEAN,TOTAL_PAID_SO_FAR_SUM,TOTAL_PAID_SO_FAR_STD,PAYMENT_REGULARITY_MEAN,PAYMENT_REGULARITY_SUM,PAYMENT_REGULARITY_STD,DELAYED_PAYMENT_COUNT_MEAN,DELAYED_PAYMENT_COUNT_SUM,DELAYED_PAYMENT_COUNT_STD,VERSION_PAYMENT_INTERACTION_MEAN,VERSION_PAYMENT_INTERACTION_SUM,VERSION_PAYMENT_INTERACTION_STD,VERSION_CHANGE_CAT_NUNIQUE,DAY_COUNT_MEAN,DAY_COUNT_SUM,DAY_COUNT_STD,SUM_LAST_180_DAYS_MEAN,SUM_LAST_180_DAYS_SUM,SUM_LAST_180_DAYS_STD
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000011,268838.0,1.000000,12.0,0.000000,6.500000,78,3.605551,-270.000000,-3240.0,108.166538,-271.666667,-3260.0,108.963158,92429.887500,1109158.650,17.848784,92429.887500,1109158.650,17.848784,0.000000,0.0,0.000000,-1.666667,-20.0,1.969464,1.0,1.000000,12.000000,0.000000,1.000000,92429.887500,1.109159e+06,9.321223,600791.69250,7209500.31,333251.994979,1.969464,23.633566,0.0,1.333333,16,0.492366,92429.887500,1109158.65,17.848784,1,2652.000000,31824.0,108.166538,600822.607500,7209871.29,333270.702459
1000019,193910.0,1.000000,13.0,0.000000,6.538462,85,3.454837,-2140.846154,-27831.0,103.645105,-2153.384615,-27994.0,105.266755,6067.201154,78873.615,17.897402,5600.111538,72801.450,1682.691565,0.000000,0.0,0.000000,-12.538462,-163.0,8.362769,2.0,0.923077,12.000000,0.277346,0.000015,5210.875962,6.774139e+04,1838.070588,36388.35000,473048.55,23622.799285,8.362769,108.715991,0.0,0.615385,8,0.506370,5600.111538,72801.45,1682.691565,1,781.153846,10155.0,103.645105,39230.570769,509997.42,20969.723379
1000025,3770.0,1.090909,12.0,0.301511,6.000000,66,3.316625,-298.000000,-3278.0,99.498744,-310.727273,-3418.0,98.041921,2850.435000,31354.785,774.896216,2850.435000,31354.785,774.896216,0.000000,0.0,0.447214,-12.727273,-140.0,6.497552,-7.0,1.000000,11.000000,0.000000,1.000000,2772.555000,3.049810e+04,346.544123,16168.05000,177848.55,9407.980354,6.497552,71.473072,0.0,0.000000,0,0.000000,3321.965455,36541.62,2338.785811,3,2624.000000,28864.0,99.498744,15934.410000,175278.51,9091.177523
1000042,62690.0,1.000000,8.0,0.000000,4.500000,36,2.449490,-1676.000000,-13408.0,73.484692,-1681.625000,-13453.0,73.853015,5085.033750,40680.270,1.240972,5085.033750,40680.270,1.240972,0.000000,0.0,0.000000,-5.625000,-45.0,2.669270,-1.0,1.000000,8.000000,0.000000,1.000000,5085.033750,4.068027e+04,0.605533,22881.99375,183055.95,12456.198642,2.669270,21.354157,0.0,0.000000,0,0.000000,5085.033750,40680.27,1.240972,1,1246.000000,9968.0,73.484692,22881.116250,183048.93,12455.379816
1000049,234821.0,3.500000,21.0,1.870829,2.166667,13,1.169045,-280.000000,-1680.0,35.071356,-290.000000,-1740.0,36.823905,8539.020000,51234.120,5831.210231,14539.020000,87234.120,6541.326026,-0.333333,-2.0,2.732520,-10.000000,-60.0,2.449490,-6.0,2.267795,13.606773,1.805770,1.000000,12274.070000,7.364442e+04,5863.110940,44463.24000,266779.44,31551.795435,2.449490,14.696938,0.0,0.000000,0,0.000000,44952.060000,269712.36,29220.299006,6,2642.000000,15852.0,35.071356,56821.080000,340926.48,26032.843949


In [14]:
# rename Sk_ID_CURR_FIRST to SK_ID_CURR
installments_agg_1.rename(columns={'SK_ID_CURR_FIRST': 'SK_ID_CURR'}, inplace=True)
installments_agg_2 = installments_agg_1.groupby('SK_ID_CURR').agg(['mean', 'sum', 'std'])

# rename columns
installments_agg_2.columns = pd.Index([e[0] + "_" + e[1].upper() for e in installments_agg_2.columns.tolist()])

installments_agg_2.head()

,NUM_INSTALMENT_VERSION_MEAN_MEAN,NUM_INSTALMENT_VERSION_MEAN_SUM,NUM_INSTALMENT_VERSION_MEAN_STD,NUM_INSTALMENT_VERSION_SUM_MEAN,NUM_INSTALMENT_VERSION_SUM_SUM,NUM_INSTALMENT_VERSION_SUM_STD,NUM_INSTALMENT_VERSION_STD_MEAN,NUM_INSTALMENT_VERSION_STD_SUM,NUM_INSTALMENT_VERSION_STD_STD,NUM_INSTALMENT_NUMBER_MEAN_MEAN,NUM_INSTALMENT_NUMBER_MEAN_SUM,NUM_INSTALMENT_NUMBER_MEAN_STD,NUM_INSTALMENT_NUMBER_SUM_MEAN,NUM_INSTALMENT_NUMBER_SUM_SUM,NUM_INSTALMENT_NUMBER_SUM_STD,NUM_INSTALMENT_NUMBER_STD_MEAN,NUM_INSTALMENT_NUMBER_STD_SUM,NUM_INSTALMENT_NUMBER_STD_STD,DAYS_INSTALMENT_MEAN_MEAN,DAYS_INSTALMENT_MEAN_SUM,DAYS_INSTALMENT_MEAN_STD,DAYS_INSTALMENT_SUM_MEAN,DAYS_INSTALMENT_SUM_SUM,DAYS_INSTALMENT_SUM_STD,DAYS_INSTALMENT_STD_MEAN,DAYS_INSTALMENT_STD_SUM,DAYS_INSTALMENT_STD_STD,DAYS_ENTRY_PAYMENT_MEAN_MEAN,DAYS_ENTRY_PAYMENT_MEAN_SUM,DAYS_ENTRY_PAYMENT_MEAN_STD,DAYS_ENTRY_PAYMENT_SUM_MEAN,DAYS_ENTRY_PAYMENT_SUM_SUM,DAYS_ENTRY_PAYMENT_SUM_STD,DAYS_ENTRY_PAYMENT_STD_MEAN,DAYS_ENTRY_PAYMENT_STD_SUM,DAYS_ENTRY_PAYMENT_STD_STD,AMT_INSTALMENT_MEAN_MEAN,AMT_INSTALMENT_MEAN_SUM,AMT_INSTALMENT_MEAN_STD,AMT_INSTALMENT_SUM_MEAN,AMT_INSTALMENT_SUM_SUM,AMT_INSTALMENT_SUM_STD,AMT_INSTALMENT_STD_MEAN,AMT_INSTALMENT_STD_SUM,AMT_INSTALMENT_STD_STD,AMT_PAYMENT_MEAN_MEAN,AMT_PAYMENT_MEAN_SUM,AMT_PAYMENT_MEAN_STD,AMT_PAYMENT_SUM_MEAN,AMT_PAYMENT_SUM_SUM,...,TOTAL_PAID_SO_FAR_STD_SUM,TOTAL_PAID_SO_FAR_STD_STD,PAYMENT_REGULARITY_MEAN_MEAN,PAYMENT_REGULARITY_MEAN_SUM,PAYMENT_REGULARITY_MEAN_STD,PAYMENT_REGULARITY_SUM_MEAN,PAYMENT_REGULARITY_SUM_SUM,PAYMENT_REGULARITY_SUM_STD,PAYMENT_REGULARITY_STD_MEAN,PAYMENT_REGULARITY_STD_SUM,PAYMENT_REGULARITY_STD_STD,DELAYED_PAYMENT_COUNT_MEAN_MEAN,DELAYED_PAYMENT_COUNT_MEAN_SUM,DELAYED_PAYMENT_COUNT_MEAN_STD,DELAYED_PAYMENT_COUNT_SUM_MEAN,DELAYED_PAYMENT_COUNT_SUM_SUM,DELAYED_PAYMENT_COUNT_SUM_STD,DELAYED_PAYMENT_COUNT_STD_MEAN,DELAYED_PAYMENT_COUNT_STD_SUM,DELAYED_PAYMENT_COUNT_STD_STD,VERSION_PAYMENT_INTERACTION_MEAN_MEAN,VERSION_PAYMENT_INTERACTION_MEAN_SUM,VERSION_PAYMENT_INTERACTION_MEAN_STD,VERSION_PAYMENT_INTERACTION_SUM_MEAN,VERSION_PAYMENT_INTERACTION_SUM_SUM,VERSION_PAYMENT_INTERACTION_SUM_STD,VERSION_PAYMENT_INTERACTION_STD_MEAN,VERSION_PAYMENT_INTERACTION_STD_SUM,VERSION_PAYMENT_INTERACTION_STD_STD,VERSION_CHANGE_CAT_NUNIQUE_MEAN,VERSION_CHANGE_CAT_NUNIQUE_SUM,VERSION_CHANGE_CAT_NUNIQUE_STD,DAY_COUNT_MEAN_MEAN,DAY_COUNT_MEAN_SUM,DAY_COUNT_MEAN_STD,DAY_COUNT_SUM_MEAN,DAY_COUNT_SUM_SUM,DAY_COUNT_SUM_STD,DAY_COUNT_STD_MEAN,DAY_COUNT_STD_SUM,DAY_COUNT_STD_STD,SUM_LAST_180_DAYS_MEAN_MEAN,SUM_LAST_180_DAYS_MEAN_SUM,SUM_LAST_180_DAYS_MEAN_STD,SUM_LAST_180_DAYS_SUM_MEAN,SUM_LAST_180_DAYS_SUM_SUM,SUM_LAST_180_DAYS_SUM_STD,SUM_LAST_180_DAYS_STD_MEAN,SUM_LAST_180_DAYS_STD_SUM,SUM_LAST_180_DAYS_STD_STD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,1.008333,4.033333,0.683333,3.25,13.0,2.500000,0.432918,1.731671,0.307497,4.275000,17.100000,4.262531,59.000000,236,102.127371,2.342536,9.370143,2.519231,-174.225000,-696.900000,133.984088,-1046.500000,-4186.0,892.563536,42.334654,169.338617,21.820181,-194.658333,-778.633333,147.253479,-1128.250000,-4513.0,928.598361,46.285659,185.142637,23.073263,17514.131812,70056.527250,15919.817110,77735.59875,310942.395,44149.780173,21746.780365,86987.121460,19847.275901,17514.131812,70056.527250,15919.817110,77735.59875,310942.395,...,118840.928285,21502.535674,9.715769,38.863077,12.162081,56.001934,224.007736,59.820937,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,28815.823125,115263.292500,31829.756403,91216.99125,364867.965,96917.351614,40910.913760,163643.655041,47457.120175,2.000000,8,0.816497,2747.775000,10991.100000,133.984088,20868.500000,83474.0,24011.566831,42.334654,169.338617,21.820181,35637.304125,142549.216500,19292.800471,3.353049e+05,1.341220e+06,4.668329e+05,34393.312986,137573.251946,21278.096586
13.0,1.050000,4.200000,0.100000,8.50,34.0,3.000000,0.111803,0.

In [15]:
installments_agg_2.to_csv('processed-data/installments_agg_2.csv')

In [16]:
app_train = pd.read_csv('processed-data/app_train.csv')
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'processed-data/app_train.csv'

In [ ]:
# merge train data with installments
app_train = app_train.merge(installments_agg_2, left_index=True, right_index=True, how='left')
app_train.head()


In [ ]:
app_train.info()

In [ ]:
# check inf values
app_train.replace([np.inf, -np.inf], np.nan, inplace=True)
app_train.isnull().sum().sum()

# fill na with 0
for col in app_train.columns:
    app_train[col].fillna(0, inplace=True)
app_train.isnull().sum().sum()

0

In [ ]:
X, y = app_train.drop('TARGET', axis=1), app_train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

gini(y_test, y_pred_proba)

0.4831450610737624

In [ ]:
app_train = pd.read_csv('processed-data/app_train.csv')
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head()

,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_QRT,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Other,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Other,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Other,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Other,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Self-employed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,1,0,0,0,2,2,0,0,0,0,0,0,0.126697,0.285180,0.0,0,1,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,1,0,0,0,3,3,0,0,0,0,0,0,0.598301,0.746300,-142.0,0,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,1,265851.0,11263.5,229500.0,0.031329,-14434,-3759,-4976.0,-3989,1,0,0,0,2,2,0,0,0,0,0,0,0.293988,0.415347,0.0,0,1,0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0,2,545040.0,20547.0,450000.0,0.004849,-15957,-6018,-10110.0,-5219,1,0,1,0,2,2,0,0,0,0,0,0,0.070575,0.397946,-725.0,0,1,0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0,0,512064.0,25033.5,360000.0,0.018801,-17851,-495,-43.0,-181,1,0,0,0,2,2,0,0,0,0,1,1,0.501790,0.529890,0.0,0,1,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
installment = process_installment(installments)
installment.head()

"Installments Payments" final shape: (97828, 85)


,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DPD_VAR,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_DBD_VAR,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,INSTAL_PAYMENT_PERC_VAR,INSTAL_PAYMENT_DIFF_MAX,INSTAL_PAYMENT_DIFF_MEAN,INSTAL_PAYMENT_DIFF_SUM,INSTAL_PAYMENT_DIFF_VAR,INSTAL_AMT_INSTALMENT_MAX,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_INSTALMENT_MIN,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MIN,INSTAL_SK_ID_PREV_SIZE,INSTAL_SK_ID_PREV_NUNIQUE,INSTAL_PAYMENT_DIFFERENCE_MEAN,INSTAL_PAYMENT_RATIO_MEAN,INSTAL_PAYMENT_RATIO_MAX,INSTAL_LATE_PAYMENT_MEAN,INSTAL_LATE_PAYMENT_SUM,INSTAL_SIGNIFICANT_LATE_PAYMENT_MEAN,INSTAL_SIGNIFICANT_LATE_PAYMENT_SUM,INSTAL_LATE_PAYMENT_RATIO_MEAN,INSTAL_DPD_7_MEAN,INSTAL_DPD_15_MEAN,INSTAL_PAID_OVER_MEAN,INSTAL_DPD_diff_MEAN,INSTAL_DPD_diff_MIN,INSTAL_DPD_diff_MAX,INSTAL_DBD_diff_MEAN,INSTAL_DBD_diff_MIN,INSTAL_DBD_diff_MAX,INSTAL_DAYS_INSTALMENT_MEAN,INSTAL_DAYS_INSTALMENT_MAX,INSTAL_DAYS_INSTALMENT_SUM,INSTAL_INS_IS_DPD_UNDER_120_MEAN,INSTAL_INS_IS_DPD_UNDER_120_SUM,INSTAL_INS_IS_DPD_OVER_120_MEAN,INSTAL_INS_IS_DPD_OVER_120_SUM,INSTAL_COUNT,INS_D365SK_ID_CURR_COUNT,INS_D365NUM_INSTALMENT_VERSION_NUNIQUE,INS_D365DAYS_ENTRY_PAYMENT_MEAN,INS_D365DAYS_ENTRY_PAYMENT_MAX,INS_D365DAYS_ENTRY_PAYMENT_SUM,INS_D365DAYS_INSTALMENT_MEAN,INS_D365DAYS_INSTALMENT_MAX,INS_D365DAYS_INSTALMENT_SUM,INS_D365AMT_INSTALMENT_MEAN,INS_D365AMT_INSTALMENT_MAX,INS_D365AMT_INSTALMENT_SUM,INS_D365AMT_PAYMENT_MEAN,INS_D365AMT_PAYMENT_MAX,INS_D365AMT_PAYMENT_SUM,INS_D365PAYMENT_DIFF_MEAN,INS_D365PAYMENT_DIFF_MIN,INS_D365PAYMENT_DIFF_MAX,INS_D365PAYMENT_DIFF_SUM,INS_D365PAYMENT_PERC_MEAN,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,3,0.0,0.000000,0.0,0.000000,64.0,10.900000,327.0,374.575862,1.00000,1.000000,30.000000,0.000000,0.00,0.000000,0.00,0.000000e+00,100287.765,10364.746500,310942.395,141.570,141.570,100287.765,10364.746500,310942.395,-5.0,-150.433333,-4513.0,-446.0,30,4,0.000000,1.000000,1.0,0.366667,11,0.366667,11,0.366667,0.0,0.0,0.000000,-10.900000,-64.0,0.0,10.900000,0.0,64.0,-139.533333,-5.0,-4186.0,0.000000,0,0.0,0,30,26.0,3.0,-107.961538,-5.0,-2807.0,-103.461538,-5.0,-2690.0,10965.515192,100287.765,285103.395,10965.515192,100287.765,285103.395,0.0,0.0,0.0,0.0,1.0,1.0,-4.500000,-33.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3.0,4,0.0,0.000000,0.0,0.000000,99.0,11.581395,498.0,459.439646,2.51059,1.050525,45.172583,0.062151,0.00,-204.069767,-8775.00,9.119528e+05,43050.285,5691.316395,244726.605,78.435,78.435,43050.285,5895.386163,253501.605,-5.0,-228.232558,-9814.0,-713.0,43,4,-612.209302,0.965116,1.0,0.511628,22,0.511628,22,0.562153,0.0,0.0,0.046512,-11.581395,-99.0,0.0,11.581395,0.0,99.0,-216.651163,-5.0,-9316.0,0.000000,0,0.0,0,43,37.0,3.0,-175.675676,-5.0,-6500.0,-166.972973,-5.0,-6178.0,5453.012432,43050.285,201761.460,5453.012432,43050.285,201761.460,0.0,0.0,0.0,0.0,1.0,1.0,-8.702703,-99.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
5.0,1,0.0,0.000000,0.0,0.000000,17.0,9.200000,92.0,13.066667,1.00000,1.000000,10.000000,0.000000,0.00,0.000000,0.00,0.000000e+00,9820.755,9818.644500,98186.445,9799.650,9799.650,9820.755,9818.644500,98186.445,-1808.0,-1944.200000,-19442.0,-2083.0,10,1,0.000000,1.000000,1.0,1.000000,10,1.000000,10,1.000000,0.0,0.0,0.000000,-9.200000,-17.0,-6.0,9.200000,6.0,17.0,-1935.000000,-1800.0,-19350.0,0.000000,0,0.0,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,2,1.0,0.

In [ ]:
installment.to_csv('processed-data/processed_installment.csv')